In [1]:
import pandas as pd
import numpy as np
import pickle
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *

In [2]:
df = pd.read_csv('A1_dataset.csv')

In [3]:
# text = df['TEXT'].to_list()
# def preprocess_text(text):
#     text = lowercase_text(text)
#     text = remove_url_html(text)
#     text = remove_users(text)
#     text = remove_punctuations(text)
#     text = remove_whitespaces(text)
#     text = tokenization(text)
#     text = spelling_correction(text, 'autocorrect_full')
#     text = remove_alphanum(text)
#     return text
# df['preprocessed_text'] = df['TEXT'].progress_apply(preprocess_text)
# preprocess_text = df['preprocessed_text'].to_list()
# df.to_csv('preprocessed_A1.csv', index=False)

In [4]:
df = pd.read_csv('preprocessed_A1.csv')
preprocess_text = df['preprocessed_text'].to_list()
for i in range(len(preprocess_text)):
    preprocess_text[i] = eval(preprocess_text[i])
    

In [5]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [6]:
unigram_probs = {}
scale_factor = sum(unigram_counts.values())
for word in unigram_counts:
    unigram_probs[word] = unigram_counts[word] / scale_factor

In [7]:
len(unigram_counts)

6952

# Saving Bigrams with Smoothing

In [8]:
# bigram_wrapper = Bigrams(preprocess_text)
# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

# with open('bigram_wrapper.pickle', 'wb') as f:
#     pickle.dump(bigram_wrapper, f)

In [9]:
# load bigram_wrapper
# with open('bigram_wrapper.pickle', 'rb') as f:
#     bigram_wrapper = pickle.load(f)

# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [10]:
# smoother = LMSmoothers(bigrams, vocab_len, unigram_counts=unigram_counts, unigrams_probs=unigram_probs)
# laplace_smoothed_bigrams = smoother.laplace()
# add_5_smoothed_bigrams = smoother.add_k(5)
# add_5_with_unigram_prior_smoothed_bigrams = smoother.add_k_with_unigram_prior(5)

In [11]:
# with open('laplace_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('add_5_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('add_5_with_unigram_prior_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_with_unigram_prior_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Creating BiGram Models

In [13]:
with open('laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

# with open('add_5_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_smoothed_bigrams = pickle.load(handle)

# with open('add_5_with_unigram_prior_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_with_unigram_prior_smoothed_bigrams = pickle.load(handle)

In [9]:
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)

In [15]:
# Load Positive Prompts
with open('pos_250_vader.txt', 'rb') as handle:
    pos_prompts = handle.readlines()

for i in range(len(pos_prompts)):
    pos_prompts[i] = pos_prompts[i].decode('utf-8').strip()

In [16]:
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
lm.generate_text(['paradise'], 1, 7)

['paradise', 'perfectly', 'freedom', 'mother', 'is', 'a', 'lot']

In [18]:
lm.computePerplexity(lm.generate_text(['paradise'], 1, 7))

2020.132004640018

In [ ]:
# clear the file
open('pos_gen.txt', 'w').close()

In [19]:
def append_to_file(filename, text1, text2):
    with open(filename, 'a') as f:
        f.write(text1 + " " + text2 + '\n')

In [20]:
ls_pos_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
for p in pos_prompts:
    sentence_pos = lm.generate_text([p], sentiment=1, length=10)
    # sentence_neg = lm.generate_text(['i', 'am'], sentiment=-1, length=10)
    print("Positive: {}".format(sentence_pos))
    # print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_pos)
    print("Positive Perplexity: {}".format(ppl))
    # print("Negative Perplexity: {}".format(lm.computePerplexity(sentence_neg)))
    ls_pos_gen.append((sentence_pos, ppl))
    append_to_file('pos_gen.txt', " ".join(sentence_pos), str(ppl))

Positive: ['paradise', 'freedom', 'greatest', 'games', 'like', 'that', 'im', 'sorry', 'to', 'see']
Positive Perplexity: 1518.033414028229
Positive: ['perfectly', 'paradise', 'look', 'like', 'to', 'the', 'same', 'time', 'for', 'a']
Positive Perplexity: 857.6415965567389
Positive: ['freedom', 'of', 'the', 'last', 'time', 'i', 'am', 'in', 'my', 'baby']
Positive Perplexity: 619.8685178267399
Positive: ['greatest', 'games', 'paradise', 'look', 'like', 'a', 'great', 'time', 'to', 'go']
Positive Perplexity: 1047.5797729105059
Positive: ['glorious', 'sunny', 'day', 'to', 'see', 'a', 'great', 'idea', 'serena', 'greatest']
Positive Perplexity: 1361.984966818199


KeyboardInterrupt: 

In [ ]:
# save generated sentences
with open('ls_pos_gen_vader_thread.pickle', 'wb') as handle:
    pickle.dump(ls_pos_gen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load Positive Prompts
with open('neg_250_vader.txt', 'rb') as handle:
    neg_prompts = handle.readlines()

for i in range(len(neg_prompts)):
    neg_prompts[i] = neg_prompts[i].decode('utf-8').strip()

In [ ]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
for n in neg_prompts:
    sentence_neg = lm.generate_text([n], sentiment=-1, length=10)
    # sentence_neg = lm.generate_text(['i', 'am'], sentiment=-1, length=10)
    print("Negative: {}".format(sentence_pos))
    # print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_pos)
    print("Negative Perplexity: {}".format(ppl))
    # print("Negative Perplexity: {}".format(lm.computePerplexity(sentence_neg)))
    ls_neg_gen.append((sentence_pos, ppl))
    append_to_file('nnn.txt', " ".join(sentence_pos), str(ppl))

In [31]:
ls = laplace_smoothed_bigrams.values()

In [33]:
ls = np.array(list(ls))

In [40]:
min(ls)

1.3853485153122987e-08